  # **AKADEMI EDUCATION**
# **Première cohorte (2025): Science des données et intelligence artificielle**
#### *Phase 5: PROJET DE SCIENCE DES DONNÉES*

**Noms des étudiants du groupe: Riché FLEURINORD et Micka LOUIS**   
**Rythme d’apprentissage: Autonome**  
**Date de soutenance: 27 octobre 2025**  
**Noms des instructeurs: Wedter JEROME et Geovany Batista Polo LAGUERRE**  
**Lien de l’article de blog (lien du dépôt GitHub): https://github.com/richefleuriord/Bank_Customer_Churn_Prediction.git**

# *4-Modélisation*

## Modélisation et Sélection du Meilleur Modèle

La **modélisation** constitue une étape centrale du projet de *Data Science*.  
Elle vise à entraîner plusieurs algorithmes de classification afin d’identifier celui qui offre la meilleure performance prédictive sur le risque de départ (**churn**) des clients bancaires.

Après avoir nettoyé, transformé et équilibré les données grâce à la phase de **préparation**, nous disposons désormais de trois jeux distincts :
- **Train (70%)** : pour l’apprentissage du modèle.  
- **Validation (15%)** : pour l’optimisation des hyperparamètres.  
- **Test (15%)** : pour l’évaluation finale de performance.

---

### Objectifs de la modélisation

- Comparer les performances de plusieurs algorithmes de *Machine Learning* (régression logistique, forêt aléatoire, XGBoost, réseau de neurones).  
- Utiliser une validation croisée (*cross-validation*) pour garantir la robustesse des modèles.  
- Optimiser les hyperparamètres à l’aide de **GridSearchCV**.  
- Identifier le modèle le plus performant selon le score **ROC-AUC**, plus fiable que l’accuracy en cas de classes déséquilibrées.  
- Créer un **pipeline complet** prêt pour le déploiement dans Streamlit.

---

### Étapes principales

1. **Chargement des données préparées** : import des fichiers `.csv` (train, validation, test).  
2. **Définition des modèles candidats** avec leurs *pipelines* respectifs (scaling, encodage, modèle).  
3. **Recherche d’hyperparamètres** via *GridSearchCV* sur chaque algorithme.  
4. **Évaluation sur le jeu de validation** et comparaison des résultats.  
5. **Sélection du meilleur modèle** selon le score ROC-AUC.  
6. **Construction du pipeline final de déploiement**.  
7. **Évaluation finale sur le jeu de test**.  
8. **Sauvegarde du modèle** prêt à être intégré dans l’application Streamlit.

---

### Script complet de modélisation

In [1]:
# ============================================================
# IMPORT DES LIBRAIRIES
# ============================================================

import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.compose import ColumnTransformer
from xgboost import XGBClassifier
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, roc_auc_score
import joblib
import os
import warnings
import logging

# Suppression des warnings et logs inutiles
warnings.filterwarnings('ignore')
joblib.logger = logging.getLogger('joblib')
joblib.logger.setLevel(logging.WARNING)

# Création du dossier "Models"
os.makedirs("Models", exist_ok=True)

# ============================================================
# CHARGEMENT DES DONNÉES PRÉPARÉES
# ============================================================

X_train = pd.read_csv("../Data/X_train_prepared.csv")
y_train = pd.read_csv("../Data/y_train_prepared.csv").values.ravel()
X_val = pd.read_csv("../Data/X_val_prepared.csv")
y_val = pd.read_csv("../Data/y_val_prepared.csv").values.ravel()
X_test = pd.read_csv("../Data/X_test_prepared.csv")
y_test = pd.read_csv("../Data/y_test_prepared.csv").values.ravel()

print(" Données préparées chargées avec succès !")

# ============================================================
# DÉFINITION DES MODÈLES ET PIPELINES
# ============================================================

pipelines = {
    "LogisticRegression": Pipeline([
        ("scaler", StandardScaler()),
        ("model", LogisticRegression(max_iter=1000, random_state=42))
    ]),
    "RandomForest": Pipeline([
        ("model", RandomForestClassifier(random_state=42))
    ]),
    "XGBoost": Pipeline([
        ("model", XGBClassifier(use_label_encoder=False, eval_metric='logloss', random_state=42))
    ]),
    "NeuralNetwork": Pipeline([
        ("scaler", StandardScaler()),
        ("model", MLPClassifier(max_iter=300, random_state=42))
    ])
}

# ============================================================
# GRILLES D’HYPERPARAMÈTRES POUR GRIDSEARCHCV
# ============================================================
param_grids = {
    "LogisticRegression": {
        "model__C": [0.01, 0.1, 1, 10],
        "model__penalty": ["l2"],
        "model__solver": ["lbfgs", "liblinear"]
    },
    "RandomForest": {
        "model__n_estimators": [100, 200],
        "model__max_depth": [5, 10, 20],
        "model__min_samples_split": [2, 5],
        "model__min_samples_leaf": [1, 2]
    },
    "XGBoost": {
        "model__n_estimators": [100, 200],
        "model__max_depth": [3, 5, 7],
        "model__learning_rate": [0.01, 0.1, 0.2],
        "model__subsample": [0.8, 1.0]
    },
    "NeuralNetwork": {
        "model__hidden_layer_sizes": [(50,), (100,), (100, 50)],
        "model__activation": ["relu", "tanh"],
        "model__alpha": [0.0001, 0.001],
        "model__learning_rate_init": [0.001, 0.01]
    }
}

# ============================================================
# GRIDSEARCHCV SUR CHAQUE MODÈLE
# ============================================================
best_models = {}
results = []

for name, pipeline in pipelines.items():
    print(f"\n Entraînement du modèle : {name}")
    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=param_grids[name],
        scoring='roc_auc',
        cv=5,
        n_jobs=-1,
        verbose=1
    )
    grid.fit(X_train, y_train)

    best_model = grid.best_estimator_
    best_models[name] = best_model

    print(f" Meilleurs paramètres {name} :", grid.best_params_)
    print(f" Score ROC-AUC (train) : {grid.best_score_:.4f}")

    # Évaluation sur le jeu de validation
    y_val_pred = best_model.predict(X_val)
    val_auc = roc_auc_score(y_val, best_model.predict_proba(X_val)[:, 1])
    val_acc = accuracy_score(y_val, y_val_pred)

    results.append({
        "Model": name,
        "BestParams": grid.best_params_,
        "Val Accuracy": val_acc,
        "Val ROC-AUC": val_auc
    })

# Résumé des performances
results_df = pd.DataFrame(results)
print("\n Résumé des performances sur le jeu de validation :")
print(results_df)

# ============================================================
# SÉLECTION DU MEILLEUR MODÈLE
# ============================================================
best_model_name = results_df.sort_values(by="Val ROC-AUC", ascending=False).iloc[0]["Model"]
final_model = best_models[best_model_name]

# ============================================================
# PIPELINE FINAL POUR LE DÉPLOIEMENT
# ============================================================

final_features = X_train.columns.tolist()

preprocessor = ColumnTransformer([
    ('scaler', StandardScaler(), final_features)
], remainder='passthrough')

deployment_pipeline = Pipeline([
    ('preprocessor', preprocessor),
    ('classifier', final_model)
])

deployment_pipeline.fit(X_train, y_train)
joblib.dump(deployment_pipeline, "Models/Best_Model_Deployment.pkl")

print(" Pipeline complet entraîné et sauvegardé avec succès pour Streamlit !")

# ============================================================
# ÉVALUATION FINALE SUR LE TEST SET
# ============================================================
y_test_pred = deployment_pipeline.predict(X_test)
y_test_proba = deployment_pipeline.predict_proba(X_test)[:, 1]

print(f"\n Meilleur modèle sélectionné : {best_model_name}")
print("\nÉvaluation finale sur le jeu de test :")
print(classification_report(y_test, y_test_pred))
print("Accuracy :", accuracy_score(y_test, y_test_pred))
print("ROC-AUC  :", roc_auc_score(y_test, y_test_proba))
print("Confusion Matrix :\n", confusion_matrix(y_test, y_test_pred))


 Données préparées chargées avec succès !

 Entraînement du modèle : LogisticRegression
Fitting 5 folds for each of 8 candidates, totalling 40 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  40 out of  40 | elapsed:    8.1s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


 Meilleurs paramètres LogisticRegression : {'model__C': 0.1, 'model__penalty': 'l2', 'model__solver': 'lbfgs'}
 Score ROC-AUC (train) : 0.8173

 Entraînement du modèle : RandomForest
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:   13.8s
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:   51.9s finished


 Meilleurs paramètres RandomForest : {'model__max_depth': 10, 'model__min_samples_leaf': 2, 'model__min_samples_split': 5, 'model__n_estimators': 200}
 Score ROC-AUC (train) : 0.8371

 Entraînement du modèle : XGBoost
Fitting 5 folds for each of 36 candidates, totalling 180 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:    5.5s
[Parallel(n_jobs=-1)]: Done 180 out of 180 | elapsed:   28.8s finished


 Meilleurs paramètres XGBoost : {'model__learning_rate': 0.1, 'model__max_depth': 3, 'model__n_estimators': 100, 'model__subsample': 0.8}
 Score ROC-AUC (train) : 0.8423

 Entraînement du modèle : NeuralNetwork
Fitting 5 folds for each of 24 candidates, totalling 120 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  42 tasks      | elapsed:  2.4min
[Parallel(n_jobs=-1)]: Done 120 out of 120 | elapsed:  7.6min finished


 Meilleurs paramètres NeuralNetwork : {'model__activation': 'relu', 'model__alpha': 0.001, 'model__hidden_layer_sizes': (50,), 'model__learning_rate_init': 0.001}
 Score ROC-AUC (train) : 0.8037

 Résumé des performances sur le jeu de validation :
                Model                                         BestParams  \
0  LogisticRegression  {'model__C': 0.1, 'model__penalty': 'l2', 'mod...   
1        RandomForest  {'model__max_depth': 10, 'model__min_samples_l...   
2             XGBoost  {'model__learning_rate': 0.1, 'model__max_dept...   
3       NeuralNetwork  {'model__activation': 'relu', 'model__alpha': ...   

   Val Accuracy  Val ROC-AUC  
0      0.838000     0.832923  
1      0.846000     0.853532  
2      0.856000     0.859956  
3      0.818667     0.814133  
 Pipeline complet entraîné et sauvegardé avec succès pour Streamlit !

 Meilleur modèle sélectionné : XGBoost

Évaluation finale sur le jeu de test :
              precision    recall  f1-score   support

           